In [77]:
using Plots, ComplexPhasePortrait, ApproxFun, SingularIntegralEquations
gr();

# M3M6: Methods of Mathematical Physics

$$
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\dx{\D x}
\def\dt{\D t}
\def\C{{\mathbb C}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\qqfor{\qquad\hbox{for}\qquad}
$$

Dr. Sheehan Olver
<br>
s.olver@imperial.ac.uk

Office Hours: 3-4pm Mondays, Huxley 6M40
<br>
Website: https://github.com/dlfivefifty/M3M6LectureNotes

# Lecture 12: Additive Riemann–Hilbert problems

1. Additive Riemann–Hilbert problem $\phi_+(x) + \phi_-(x) = f(x)$
   - General solution on $[-1,1]$ built _from_ a Cauchy transform
   - General solution on $[-1,1]$ expressed _as_ a Cauchy transform
2. Inverting the Hilbert transform

## Additive Riemann–Hilbert problem

We now consider an additive Riemann–Hilbert problem
$$
\phi_+(x) + \phi_-(x) = f(x) \qquad \phi(\infty) = 0
$$
and take $\Gamma = [-1,1]$. 


### General solution on $[-1,1]$ built _from_ a Cauchy transform


Here we propose the general solution to an additive Riemann–Hilbert problem:

**Theorem (Cauchy transform solution to Additive Riemann–Hilbert problem)** Suppose $\phi(z)$ analytic in $\C \backslash [-1,1]$ satisfies

1. $\phi_+(x) + \phi_-(x) = f(x) $ for $-1 < x < 1$, where $f$ is smooth on $[-1,1]$.
2. $\phi(z)$ has weaker than pole singularities,
3. $\phi(\infty) = 0$.

Then, for some constant $C$, 
$$
 \phi(z) = {\I \over \sqrt{z-1}\sqrt{z+1}} \CC_{[-1,1]} \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](z)  + {C \over \sqrt{z-1} \sqrt{z+1}}
$$


In [2]:
x = Fun()
f = exp(x)
v = f*sqrt(1-x^2)

C = randn() # doesn't matter

φ = z -> im*cauchy(v, z)/(sqrt(z-1)sqrt(1+z)) + C/(sqrt(z-1)sqrt(z+1))

@show φ(1.0E8)
@show f(0.1)
@show φ(0.1+0.0im) + φ(0.1-0.0im);

phaseplot(φ, (-3,3), (-2,2))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2

On other intervals $(a,b)$, we get the same forms of solution:

$$
 \phi(z) = {\I \over \sqrt{z-b}\sqrt{z-a}} \CC_{[a,b]} \left[{ f(\diamond)  \sqrt{b-\diamond}\sqrt{\diamond-a} }\right](z)  + {C \over \sqrt{z-b} \sqrt{z-a}}
$$

### General solution on $[-1,1]$ _expressed as_ a Cauchy transform

Note the expression above is _not_ a Cauchy transform of a function. We can determine a Cauchy transform expression by taking the difference: on $[-1,1]$ we have

\begin{align*}
    u(x) &= \phi_+(x) - \phi_-(x) = {1 \over \sqrt{1 - x^2}}\left(        \CC_{[-1,1]}^+ + \CC_{[-1,1]}^- \right) \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](x) - {2C \I \over \sqrt{1-x^2}} \\
    &= {-\I \over \sqrt{1 - x^2}}\HH_{[-1,1]} \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](x) - {2C \I \over \sqrt{1-x^2}}
\end{align*}
BY (**Plemelj II**), we guarantee that 
$$
\phi(z) = \CC_{[-1,1]} u(z)
$$

In [27]:
x = Fun()
f = exp(x)
v = f*sqrt(1-x^2)

C = randn() # doesn't matter

φ = z -> im*cauchy(v, z)/(sqrt(z-1)sqrt(1+z)) + C/(sqrt(z-1)sqrt(z+1))

u = -im/sqrt(1-x^2)*hilbert(f*sqrt(1-x^2)) - 2*C*im/sqrt(1-x^2)

@show φ(1.0+2.0im);
@show cauchy(u, 1.0+2.0im);

φ(1.0 + 2.0im) = 0.2075817508091346 - 0.36825812558937665im
cauchy(u, 1.0 + 2.0im) = 0.20758175080913466 - 0.3682581255893767im


On other intervals we have
\begin{align*}
    u(x) &= {-\I \over \sqrt{b - x}\sqrt{x-a}}\HH_{[a,b]} \left[{ f(\diamond)  \sqrt{b-\diamond}\sqrt{\diamond-a} }\right](x) - {2C \I \over \sqrt{b-x} \sqrt{x-a}}
\end{align*}


In the special case where we can calculate 
$\CC \left[ f(\diamond) \sqrt{b-\diamond}\sqrt{\diamond-a}\right](z)$ exactly, we can work out the precise formula for $u(x)$ by taking the difference.


**Example** Consider solving

1. $\phi_+(x) + \phi_-(x) = x $ for $-b < x < b$
2. $\phi(z)$ has weaker than pole singularities,
3. $\phi(\infty) = 0$.

Note that 
$$
    \kappa(z) = {z \sqrt{z-b} \sqrt{z+b} - z^2 + b^2/2 \over 2 \I} 
$$
satisfies
$$
\kappa^+(x) - \kappa^-(x) = x \sqrt{b^2 - x^2}
$$
and $\kappa(\infty) = 0$. The second property follows since $\kappa$ is analytic off $[-1,1]$ and has at most polynomial growth at $\infty$.  To understand the order of the polynomial growth, it's sufficient to study the limit on the real axis as $x \rightarrow \infty$, where we use the Taylor series of $\sqrt x$ near $1$ to determine:
\begin{align*}
x \sqrt{x-b}\sqrt{x + b} &= x^2 \sqrt{1-b/x} \sqrt{1 + b/x} = x^2 \left(1 - {b\over 2x} - {b^2 \over 8x^2}\right) \left(1 + {b\over 2x} - {b^2 \over 8x^2}\right)\\
   = x^2 - {b^2\over 2}  + O(x^{-2})
\end{align*}

In [29]:
b = 2
κ = z -> (z*sqrt(z-b)*sqrt(z+b) - z^2 + b^2/2)/(2im)
@show κ(1E5)
κ(0.1+0.0im)- κ(0.1-0.0im),   0.1sqrt(b^2-0.1^2)

κ(100000.0) = 0.0 - 0.0im


(0.1997498435543818 + 0.0im, 0.1997498435543818)

Hence every solution has the form
\begin{align*}
\phi(z) = {1 \over \sqrt{z-b} \sqrt{z+b}} \left({z \sqrt{z-b} \sqrt{z+b} - z^2 + b^2/2 \over 2 }  \right) + {C \over \sqrt{z-b} \sqrt{z+b}} \\
=  {z  \over 2 } - {z^2 \over 2  \sqrt{z-b} \sqrt{z+b}} + {D \over \sqrt{z-b} \sqrt{z+b}}
\end{align*}
where $D$ is again a arbitrary constant.

In [41]:
D = randn()

φ = z -> z/(2) - z^2/(2*sqrt(z-b)*sqrt(z+b)) + D/(sqrt(z-b)*sqrt(z+b))
φ(0.1+0.0im) + φ(0.1-0.0im)

0.1 + 0.0im

Inspection reveals that
$$
u(x) = \phi_+(x) - \phi_-(x) = \I {x^2 - 2D \over  \sqrt{b^2-x^2}}
$$

In [46]:
u = im*(x^2 - 2D)/sqrt(b^2 - x^2)
@show cauchy(u ,2+2im)
@show φ(2+2im);

cauchy(u, 2 + 2im) = -0.3063455117667045 + 0.5696721400612655im
φ(2 + 2im) = -0.3063455117667043 + 0.5696721400612657im


Therefore, $\CC u(z)$ satsifies $\CC^+u(x) + \CC^- u(x) = x$:

In [48]:
cauchy(u, 0.1+0.0im) + cauchy(u, 0.1-0.0im)

0.10000000000000002 + 0.0im

$D$ is a free paremeter, hence, similar to ODEs, we can add a boundary condition. In other words we can ask for the solution satisfying, for example $u(0) = 0$. This is precisely when $D = 0$ and $u(x) = \I x^2 / \sqrt{b^2-x^2}$.

In [57]:
D = 0
u = im*(x^2 - 2D)/sqrt(b^2 - x^2)
plot(imag(u); ylims=(-5,5), yaxis="Im", label="u s.t. u(0) = 0")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 Im 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 u s.t. u(0) = 0

Often we ask for the solution that is bounded at the left/right endpoint. Because of the symmetry in the problem (since $x$ is an odd function), we can actually get a solution that is bounded at both $\pm b$ by choosing $D =  b^2/2$:
$$
\phi_+(x) - \phi_-(x) = -\I \sqrt{b^2 - x^2}
$$

In [58]:
b = 2.3
x = Fun(-b .. b)
u = -im*sqrt(b^2 - x^2)
@show cauchy(u, 0.1+0.0im) + cauchy(u, 0.1-0.0im)
plot(imag(u); label="u", yaxis="Im")

cauchy(u, 0.1 + 0.0im) + cauchy(u, 0.1 - 0.0im) = 0.1 + 0.0im


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -2.0 
 
 
 -1.5 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 Im 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 u

In other words,
$$
\phi(z) = - \I \CC_{[-b,b]} \sqrt{b^2 - \diamond^2}(z)
$$
is a bounded solution to the additive Riemann–Hilbert problem.

## Inverting the Hilbert transform

We can use this along with Plemelj's lemma to invert the Hilbert transform. That is, we want to find $v(x)$ such that
$$
\HH v(x) = f(x)
$$
Based on the above construction of $u(x)$, consider
$$
    v(x) =-\I u(x) =  {-1 \over \sqrt{1 - x^2}}\HH_{[-1,1]} \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](x) - {2C \over \sqrt{1-x^2}}
$$  

In [35]:
f = exp(x)

C = randn()
v = -1/sqrt(1-x^2)*hilbert(f*sqrt(1-x^2)) - 2*C/sqrt(1-x^2)
hilbert(v, 0.1) - exp(0.1)

-2.220446049250313e-16

This construction works since $\phi(z) = \CC u(z) = \I \CC v(z)$, and
$$
\HH v(x) = \I (\CC^+ + \CC^-) v(x) = \phi^+(x) + \phi^-(x) = f(x)
$$

** Example** 

Consider $\HH_{[-b,b]}  v(x) = x$. We have 
$$
- \I \HH_{[-b,b]} \sqrt{b^2 - \diamond^2}(x) = \CC_{[-b,b]}^+ \sqrt{b^2 - \diamond^2}(z) + \CC_{[-b,b]}^- \sqrt{b^2 - \diamond^2}(z) = \I (\phi^+(x) + \phi^-(x) ) = \I x
$$
which shows that the solution is
$$
v(x)  = - \sqrt{b^2 - \diamond^2}
$$

In [38]:
x = Fun(-b .. b)
v = -sqrt(b^2 - x^2)

plot(v; label = "v")
plot!(hilbert(v); ylims=(-3,3), label="H[v]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 v 
 
 
 
 H[v]